In [1]:
import pandas as pd
import numpy as np
import argparse
import re
import os
import sys
sys.path.append('/home/ec2-user/CpGPython/code/')
import get_winid

In [2]:
def read_bed(file):
    bed = pd.read_csv(file,usecols=[0,1,2,5],header=None,names=['chr','pos1','pos2','strand'],sep='\s+')
    bed['coordinate'] = np.where(bed['strand']=='+',bed['pos1'],bed['pos2'])
    bed.drop(['pos1','pos2'],axis=1,inplace=True)
    bed['count'] = 1
#    bed_counts = bed.groupby(['chr','coordinate']).aggregate({'count':sum})
    return bed

In [3]:
data_dir = '/home/ec2-user/extra_storage/CpG_EWAS/ATAC/'
sites_file = '/home/ec2-user/CpGPython/data/all_sites_winid.csv'
win_path = '/home/ec2-user/CpGPython/data/wins.txt'
additional_feature_file = '/home/ec2-user/CpGPython/data/features/addtional_features'

In [4]:
all_sites = pd.read_csv(sites_file)
all_sites = get_winid.convert_chr_to_num(all_sites)
chrs = all_sites['chr'].unique()
wins = get_winid.read_wins(win_path,chrs)
files = os.listdir(data_dir)
pattern = '.*\.bed$'
reg = re.compile(pattern)
files = [f for f in files if len(reg.findall(f))>0]

/home/ec2-user/CpGPython/code/get_winid.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['chr'] = data['chr'].astype('i8')


In [5]:
files

['ENCFF678AIF.bed',
 'ENCFF784DSJ.bed',
 'ENCFF628MCV.bed',
 'ENCFF778VJL.bed',
 'ENCFF782KTJ.bed',
 'ENCFF294ZCT.bed',
 'ENCFF585YAB.bed',
 'ENCFF654ZNI.bed',
 'ENCFF845HFA.bed',
 'ENCFF765DRX.bed',
 'ENCFF608BWH.bed',
 'ENCFF846JID.bed',
 'ENCFF710ELD.bed',
 'ENCFF069SZE.bed',
 'ENCFF736KFN.bed',
 'ENCFF374OSU.bed',
 'ENCFF675QOY.bed',
 'ENCFF436NOT.bed',
 'ENCFF114WRE.bed',
 'ENCFF325VEG.bed',
 'ENCFF388DNI.bed',
 'ENCFF733PNQ.bed',
 'ENCFF799HAR.bed',
 'ENCFF739KYO.bed',
 'ENCFF656OYT.bed',
 'ENCFF049DWX.bed',
 'ENCFF787DUW.bed',
 'ENCFF205KDV.bed',
 'ENCFF135RTS.bed',
 'ENCFF159RKV.bed',
 'ENCFF829XME.bed',
 'ENCFF832AIE.bed',
 'ENCFF310VHO.bed',
 'ENCFF482CAN.bed',
 'ENCFF607XLB.bed',
 'ENCFF105ASY.bed',
 'ENCFF377DAO.bed',
 'ENCFF912HGK.bed',
 'ENCFF374VWZ.bed',
 'ENCFF305PRP.bed',
 'ENCFF885IPV.bed',
 'ENCFF780JBA.bed',
 'ENCFF670GFY.bed',
 'ENCFF293BRN.bed',
 'ENCFF469OER.bed',
 'ENCFF372HEU.bed',
 'ENCFF168OTV.bed',
 'ENCFF111ULL.bed',
 'ENCFF984OPE.bed',
 'ENCFF588PIS.bed',


In [6]:
counts_at_targets = pd.DataFrame(all_sites['winid'])
for file in files:
    bed = read_bed(data_dir+file)
    bed = get_winid.convert_chr_to_num(bed,chrs)
    bed = get_winid.get_winid(wins,bed).sort_values(['winid'])
    bed_counts = bed.groupby(['winid']).aggregate({'count':sum}).reset_index()
    counts_at_targets = pd.merge(counts_at_targets,bed_counts,on=['winid'],how='left')
    counts_at_targets['count'].fillna(0,inplace=True)
    counts_at_targets.rename(columns={'count':file[:-4]+'_ATAC_counts'},inplace=True)
    print(file+' is done')


/home/ec2-user/CpGPython/code/get_winid.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['chr'] = data['chr'].astype('i8')


ENCFF678AIF.bed is done


KeyboardInterrupt: 

In [ ]:
with pd.HDFStore(additional_feature_file,'a') as h5s:
    h5s['ATAC'] = counts_at_targets

In [12]:
counts_at_targets

,winid,ENCFF678AIF_counts,ENCFF784DSJ_counts,ENCFF628MCV_counts,ENCFF778VJL_counts,ENCFF782KTJ_counts,ENCFF294ZCT_counts,ENCFF585YAB_counts,ENCFF654ZNI_counts,ENCFF845HFA_counts,...,ENCFF907NVD_counts,ENCFF468QNB_counts,ENCFF592ALR_counts,ENCFF764CQI_counts,ENCFF906UQF_counts,ENCFF768UMR_counts,ENCFF858PLS_counts,ENCFF912PKL_counts,ENCFF217OUE_counts,ENCFF478ARL_counts
0,4198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0
1,4792,3.0,2.0,8.0,4.0,5.0,12.0,13.0,2.0,7.0,...,5.0,2.0,2.0,2.0,4.0,8.0,12.0,8.0,2.0,8.0
2,4823,14.0,7.0,13.0,13.0,16.0,25.0,13.0,10.0,23.0,...,22.0,12.0,11.0,26.0,8.0,14.0,7.0,11.0,30.0,13.0
3,4911,31.0,29.0,16.0,28.0,34.0,13.0,11.0,15.0,32.0,...,28.0,11.0,31.0,29.0,45.0,91.0,284.0,33.0,24.0,463.0
4,4912,29.0,24.0,12.0,19.0,26.0,21.0,11.0,19.0,22.0,...,21.0,12.0,20.0,27.0,25.0,90.0,228.0,17.0,17.0,490.0
5,4917,29.0,26.0,10.0,15.0,10.0,13.0,19.0,30.0,9.0,...,37.0,19.0,43.0,49.0,14.0,21.0,71.0,12.0,24.0,182.0
6,5039,17.0,7.0,6.0,2.0,5.0,8.0,2.0,3.0,6.0,...,3.0,4.0,5.0,7.0,0.0,5.0,5.0,6.0,11.0,3.0
7,5107,70.0,30.0,21.0,43.0,17.0,29.0,71.0,81.0,107.0,...,84.0,46.0,76.0,92.0,17.0,14.0,32.0,67.0,48.0,18.0
8,5336,41.0,30.0,16.0,11.0,15.0,13.0,13.0,36.0,21.0,...,37.0,18.0,36.0,55.0,4.0,7.0,6.0,9.0,45.0,11.0
9,5515,0.0,3.0,5.0,5.0,4.0,3.0,6.0,0.0,8.0,...,1.0,0.0,0.0,1.0,6.0,4.0,3.0,5.0,1.0,6.0
